In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import math
import argparse
import IPython.display as display
import matplotlib.pyplot as plt
from PIL import Image
import os, sys, shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split

## EDA

In [4]:
def readDatasetDF(datasetType, rootDir='data/'):
    """
    dataset Type: "train", "val", "test" 
    Return:
        dataframe with image name/label
    """
    df = pd.read_csv('{}/{}-calibrated-shuffled.txt'.format(rootDir, datasetType), header=None, delimiter = " ")
    df[0] = rootDir+"/"+df[0]
    return df

df_tr = readDatasetDF('train')
df_val = readDatasetDF('val')
df_test = readDatasetDF('test')
df_tr.head()

,0,1
0,data//calibrated/0077ML0005780000102730I01_DRC...,15
1,data//calibrated/0072MR0005610170103642E01_DRC...,8
2,data//calibrated/0069MR0004130000103477I01_DRC...,21
3,data//calibrated/0154ML0008510010104492E01_DRC...,8
4,data//calibrated/0019MR0000530000100138C00_DRC...,8


In [5]:
print(df_tr.shape)
print(df_val.shape)
print(df_test.shape)

(3746, 2)
(1640, 2)
(1305, 2)


In [6]:
df_tr

,0,1
0,data//calibrated/0077ML0005780000102730I01_DRC...,15
1,data//calibrated/0072MR0005610170103642E01_DRC...,8
2,data//calibrated/0069MR0004130000103477I01_DRC...,21
3,data//calibrated/0154ML0008510010104492E01_DRC...,8
4,data//calibrated/0019MR0000530000100138C00_DRC...,8
...,...,...
3741,data//calibrated/0163ML0008760050104602D01_DRC...,10
3742,data//calibrated/0072MR0005620000103655E01_DRC...,8
3743,data//calibrated/0066ML0003650000102517M00_DRC...,21
3744,data//calibrated/0157ML0008550020104531I01_DRC...,8


In [7]:
frames = [df_tr, df_val, df_test]
result = pd.concat(frames)

In [8]:
result[1].value_counts()

8     2684
24     998
5      506
9      371
10     277
21     200
23     193
3      176
7      162
14     153
17     137
12     117
15     111
16      86
0       80
20      71
19      68
13      68
6       66
4       59
2       36
11      26
1       24
18      22
Name: 1, dtype: int64

In [9]:
result.reset_index(drop = True, inplace = True)

In [10]:
result

,0,1
0,data//calibrated/0077ML0005780000102730I01_DRC...,15
1,data//calibrated/0072MR0005610170103642E01_DRC...,8
2,data//calibrated/0069MR0004130000103477I01_DRC...,21
3,data//calibrated/0154ML0008510010104492E01_DRC...,8
4,data//calibrated/0019MR0000530000100138C00_DRC...,8
...,...,...
6686,data//calibrated/0571MH0002590000201894I01_DRC...,24
6687,data//calibrated/0840ML0037090000401385I01_DRC...,17
6688,data//calibrated/0868MH0003900000302200I01_DRC...,10
6689,data//calibrated/0568MH0002630000201882E01_DRC...,24


In [11]:
#omitted classes with less than 80 samples
omit = [0, 20, 19, 13, 6, 4, 2, 11, 1, 18]
omit_index = []
for index, row in result.iterrows():
    if row[1] in omit:
        omit_index.append(index)

In [12]:
result.drop([x for x in omit_index], inplace = True, axis = 0)

In [13]:
result.reset_index(drop = True, inplace = True)

In [14]:
result

,0,1
0,data//calibrated/0077ML0005780000102730I01_DRC...,15
1,data//calibrated/0072MR0005610170103642E01_DRC...,8
2,data//calibrated/0069MR0004130000103477I01_DRC...,21
3,data//calibrated/0154ML0008510010104492E01_DRC...,8
4,data//calibrated/0019MR0000530000100138C00_DRC...,8
...,...,...
6166,data//calibrated/0571MH0002590000201894I01_DRC...,24
6167,data//calibrated/0840ML0037090000401385I01_DRC...,17
6168,data//calibrated/0868MH0003900000302200I01_DRC...,10
6169,data//calibrated/0568MH0002630000201882E01_DRC...,24


In [15]:
train_length = 0.6 * result.shape[0]
val_length = 0.3 * result.shape[0]
test_length = 0.1 * result.shape[0]

In [16]:
result["2"] = result.index
result["3"] = result[1]
result = result.drop([1], axis = 1)

In [17]:
result

,0,2,3
0,data//calibrated/0077ML0005780000102730I01_DRC...,0,15
1,data//calibrated/0072MR0005610170103642E01_DRC...,1,8
2,data//calibrated/0069MR0004130000103477I01_DRC...,2,21
3,data//calibrated/0154ML0008510010104492E01_DRC...,3,8
4,data//calibrated/0019MR0000530000100138C00_DRC...,4,8
...,...,...,...
6166,data//calibrated/0571MH0002590000201894I01_DRC...,6166,24
6167,data//calibrated/0840ML0037090000401385I01_DRC...,6167,17
6168,data//calibrated/0868MH0003900000302200I01_DRC...,6168,10
6169,data//calibrated/0568MH0002630000201882E01_DRC...,6169,24


In [18]:
X = result["2"]
Y = result["3"]

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify = Y)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1, stratify = Y_train)

In [20]:
train = pd.DataFrame(columns = ["1", "2"])
test = pd.DataFrame(columns = ["1", "2"])
val = pd.DataFrame(columns = ["1", "2"])

In [21]:
train["1"] = X_train
train["2"] = Y_train

In [22]:
val["1"] = X_val
val["2"] = Y_val

In [23]:
test["1"] = X_test
test["2"] = Y_test

In [24]:
# train = train.reset_index(drop = True, inplace = True)
# val = val.reset_index(drop = True, inplace = True)
# test = test.reset_index(drop = True, inplace = True)
train_end = 1183
val_end = 4701
test_end = 2742

In [25]:
new_df = [train, val, test]
new_result = pd.concat(new_df)

In [26]:
new_result.reset_index(drop = True, inplace = True)

In [27]:
new_result["3"] = 0

In [28]:
new_result

,1,2,3
0,541,8,0
1,948,8,0
2,2368,8,0
3,325,8,0
4,5636,12,0
...,...,...,...
6166,2247,9,0
6167,1868,8,0
6168,1518,9,0
6169,2998,8,0


In [29]:
for i in range(new_result.shape[0]):
    for index, row in new_result.iterrows():
        index1 = row["1"]
        new_result.at[row, "3"] = result.iloc[index1, 0]

KeyError: "['data//calibrated/0079MR0004480000104005I01_DRCL.JPG'] not in index"

In [ ]:
new_result

In [ ]:
class_names = pd.read_csv('data/msl_synset_words-indexed.txt', delimiter="      ", header=None, engine='python').set_index(0).to_dict()[1]
for i in class_names.keys():
    class_names[i] = class_names[i].strip()
class_names

In [ ]:
img_paths = df_tr.iloc[:, 0].tolist()
rootDir = 'msl-images/'
img = cv2.imread(img_paths[0])

In [ ]:
df_tr.iloc[:, 0].tolist()[0]

In [ ]:
plt.imshow(img)

In [ ]:
# import subplots
# for x in range(5):
#     img = cv2.imread(img_paths[x])
#     plt.imshow(img)

## Preprocessing

In [ ]:
batch_size = 32
img_height = 256
img_width = 256
path = 'data/'

train_dir = path + '/train-calibrated-shuffled.txt'
validation_dir = path + '/val-calibrated-shuffled.txt'
test_dir = path + '/test-calibrated-shuffled.txt'

def read_dataset(dir):
    df = pd.read_csv(dir, header=None, delimiter = " ")
    df[0] = path + df[0]
    image_paths = df[0].values
    labels = df[1].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    return dataset

train_ds = read_dataset(train_dir)
val_ds = read_dataset(validation_dir)
test_ds = read_dataset(test_dir)

def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    #image = (image / 255.0)
    return image, label 

def augment(image_label, seed): 
    image_path, label = image_label
    image, label = read_image(image_path, label)
    image = tf.image.resize_with_crop_or_pad(image, img_height + 6, img_width + 6)
    # Make a new seed
    new_seed = tf.random.experimental.stateless_split(seed, num=1)[0, :]
    # Random crop back to the original size
    image = tf.image.stateless_random_crop(
        image, size=[img_height, img_width, 3], seed=seed)
    # Random brightness
    image = tf.image.stateless_random_brightness(
        image, max_delta=0.5, seed=new_seed)
    image = tf.image.random_brightness(image, max_delta= 0.05)
    image = tf.image.random_hue(image, 0.08)
    image = tf.image.random_saturation(image, 0.6, 1.6)
    image = tf.image.random_contrast(image, 0.7, 1.3)
    #image = tf.clip_by_value(image, 0, 1)
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

count_train = df_tr.shape[0]
count_val = df_val.shape[0]
count_test = df_test.shape[0]
num_train = 0
num_val = 0
num_test = 0
for x in omit_index:
    if x < count_train:
        num_train +=1
    elif x > count_train and x < count_train + count_val:
        num_val += 1
    else:
        num_test +=1
        
train_ds_omit = list(train_ds.as_numpy_iterator())
val_ds_omit = list(val_ds.as_numpy_iterator())
test_ds_omit = list(test_ds.as_numpy_iterator())

# train_ds_omit = tf.data.Dataset.from_tensor_slices(train_ds_omit)
# val_ds_omit = tf.data.Dataset.from_tensor_slices(val_ds_omit)
# test_ds_omit = tf.data.Dataset.from_tensor_slices(test_ds_omit)

# frames = [train_ds_omit, val_ds_omit, test_ds_omit]
# main = pd.concat(frames)
# counter = 0
# for x in main:
#     if counter in omit_index:
#         main.drop(x, axis = 0)
#     counter +=1

# train_ds_omit = main[:count_train - num_train]
# val_ds_omit = main[count_train - num_train:(count_train - num_train) + ((count_train + count_val) - num_val)]
# test_ds_omit = main[(count_train - num_train) + ((count_train + count_val) - num_val): ]

# train_ds_omit = tf.data.Dataset.zip((train_ds_omit, (counter, counter)))
# train_ds_omit = train_ds_omit.map(augment, num_parallel_calls=AUTOTUNE)
# val_ds_omit = val_ds_omit.map(read_image, num_parallel_calls=AUTOTUNE)
# test_ds_omit = test_ds_omit.map(read_image, num_parallel_calls=AUTOTUNE)

# counter = tf.data.experimental.Counter()
# train_ds = tf.data.Dataset.zip((train_ds, (counter, counter)))
# train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
# val_ds = val_ds.map(read_image, num_parallel_calls=AUTOTUNE)
# test_ds = test_ds.map(read_image, num_parallel_calls=AUTOTUNE)

In [ ]:
train_ds_omit

In [ ]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds =  configure_for_performance(test_ds)

In [ ]:
train_ds_omit = configure_for_performance(train_ds_omit)
val_ds_omit = configure_for_performance(val_ds_omit)
test_ds_omit =  configure_for_performance(test_ds_omit)

## Generate TFRecords

In [ ]:
# display.display(display.Image(filename=df_tr[0][0]))

In [ ]:
# def _bytes_feature(value):
#     return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# def _float_feature(value):
#     return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

# def _int64_feature(value):
#     return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
# def create_tf_example(filename, label):
#     image_string = open('data/resized'+filename[16:], 'rb').read()
    
#     feature = {
#         'image': _bytes_feature(image_string),          
#         'label': _int64_feature(label),
#     }
#     tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
#     return tf_example

In [ ]:
# for x in df_tr.itertuples():
#     filename = x._1
#     print('data/resized'+filename[16:])
#     break

In [ ]:
# # Write tfrecord for training data
# writer = tf.io.TFRecordWriter("data/out/train.tfrecords")
    
# for x in df_tr.itertuples():
#     filename = x._1
#     label = x._2
#     tf_example = create_tf_example(filename, label)
#     writer.write(tf_example.SerializeToString())

In [ ]:
# # Write tfrecord for validation data
# writer = tf.io.TFRecordWriter("data/out/val.tfrecords")
    
# for x in df_val.itertuples():
#     filename = x._1
#     label = x._2
#     tf_example = create_tf_example(filename, label)
#     writer.write(tf_example.SerializeToString())

In [ ]:
# # Write tfrecord for test data
# writer = tf.io.TFRecordWriter("data/out/test.tfrecords")
    
# for x in df_test.itertuples():
#     filename = x._1
#     label = x._2
#     tf_example = create_tf_example(filename, label)
#     writer.write(tf_example.SerializeToString())

## Extract Tfrecord

In [ ]:
# # Decode the image
# def decode_image(image):
#     image = tf.image.decode_jpeg(image, channels=3)
#     image = tf.cast(image, tf.float32)
#     image = tf.reshape(image, [RESIZE, RESIZE, 3])
#     return image

# # Extract features from tfrecord
# def _extract_fn(tfrecord):
#     feature = {
#         'image': tf.io.FixedLenFeature([], tf.string),
#         'label': tf.io.FixedLenFeature([], tf.int64),
#     }
#     sample = tf.io.parse_single_example(tfrecord, feature)
#     image = decode_image(sample['image'])
#     label = tf.cast(sample['label'], tf.int32)
    
#     return [image, label]

In [ ]:
# # Batch dataset
# def batch_dataset(filename, batch_size):
#     dataset = tf.data.TFRecordDataset(filename)
#     dataset = dataset.map(_extract_fn)
#     dataset = dataset.batch(batch_size, drop_remainder=True)
#     return dataset

In [ ]:
# BATCH_SIZE = 16

# train_ds = batch_dataset('data/out/train.tfrecords', BATCH_SIZE)
# val_ds = batch_dataset('data/out/val.tfrecords', BATCH_SIZE)
# test_ds = batch_dataset('data/out/test.tfrecords', BATCH_SIZE)

In [ ]:
# image_batch, label_batch = next(iter(train_ds))

In [ ]:
# def show_batch(image_batch, label_batch):
#     plt.figure(figsize=(10, 10))
#     for n in range(25):
#         ax = plt.subplot(5, 5, n + 1)
#         plt.imshow(image_batch[n] / 255.0)
#         plt.title(class_names[label_batch[n]])
#         plt.axis("off")


# show_batch(image_batch.numpy(), label_batch.numpy())

## Transfer Learning

In [ ]:
# Baseline Resnet50 model
base_model = tf.keras.applications.ResNet50(
    include_top=False, input_shape=(64, 64, 3), weights='imagenet'
)

In [ ]:
# Freeze the base
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
num_classes = 25

inputs = tf.keras.layers.Input([64, 64, 3])
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(25, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
initial_epochs = 10

#history = model.fit(train_ds, epochs=initial_epochs, validation_data=val_ds)
history = model.fit(train_ds_omit, epochs=initial_epochs, validation_data=val_ds_omit)

In [ ]:
# Learning CUrve
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,5.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Model with image augmentation

In [ ]:
RESIZE = 255
BATCH_SIZE = 32

In [ ]:
data_augmentation = tf.keras.Sequential([        
    tf.keras.layers.experimental.preprocessing.Rescaling(1./RESIZE),                          
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

In [ ]:
epochs = 100
input_shape = (256,256,3)
# Create base model
base_model = tf.keras.applications.ResNet50V2(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
    pooling=None,
    classes=25,
    classifier_activation="softmax",
)
# Freeze base model
#base_model.trainable = False
base_model.trainable = True

fine_tune_at = 130

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
# Create new model on top.
inputs = tf.keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.resnet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(25)(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
base_model.summary()

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# history = model.fit(
#     train_ds, 
#     epochs= 10, 
#     batch_size=BATCH_SIZE, 
#     steps_per_epoch=None, 
#     validation_data=val_ds, 
#     validation_steps=None
# ) 
history = model.fit(
    train_ds_omit, 
    epochs= 10, 
    batch_size=BATCH_SIZE, 
    steps_per_epoch=None, 
    validation_data=val_ds_omit, 
    validation_steps=None
) 

In [ ]:
# Learning CUrve
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,5.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# pred = model.predict(test_ds)
pred = model.predict(test_ds_omit)
pred_class = np.argmax(pred, axis=1)

In [ ]:
# test_label = np.concatenate([y for x, y in test_ds], axis=0)
test_label = np.concatenate([y for x, y in test_ds_omit], axis=0)

In [ ]:
cf_matrix = confusion_matrix(test_label, pred_class)

In [ ]:
plt.figure(figsize=(13,13))
plt.imshow(cf_matrix, cmap=plt.cm.Blues)

for i in range(23):
    for j in range(23):
        c = cf_matrix[j,i]
        plt.text(i, j, str(c), va='center', ha='center')

ax = plt.xticks(range(22), np.sort(df_test[1].unique()))
ax = plt.yticks(range(22), np.sort(df_test[1].unique()))

## EfficientNet

In [ ]:
model = tf.keras.Sequential([
            efn.EfficientNetB0(
                input_shape=(256, 256, 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(25, activation='softmax')
])

In [ ]:
base_learning_rate = 0.0001
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [ ]:
print("Number of layers in the base model: ", len(model.layers))

In [ ]:
len(model.trainable_variables)

In [ ]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

In [ ]:
# history = model.fit(train_ds, epochs= 20, callbacks=lr_reducer, validation_data=val_ds)
history = model.fit(train_ds_omit, epochs= 20, callbacks=lr_reducer, validation_data=val_ds_omit)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()